# <h1 style="text-align: center; font-size: 3em;">Script for PyLasm</h1>

## <h2 style="text-align: center;">Multiple runs with different external magnetic field</h2>

First of all, the following python modules must be imported:

In [40]:
import os
import numpy as np
import shutil
import ast
import json
import matplotlib.pyplot as plt
import subprocess

Now, let's assign the values to some input variables according to your specific needs. Please find below a brief description for each of them:

> - **main_path** = Full path to main.py within your specific PyLasm installation directory;
> - **B_scalings** = Array of the equally-spaced values of the external magnetic field;
> - **spin_pairs** = List of 2-dim. tuples indicating the spin-spin correlation values to extract from the output files;
> - **SSC_type** = String for the type of spin-spin correlations the user is interested in (i.e. 'X components','Y components','Z components' or 'Total').

In [41]:
main_path = r"full/path/to/pylasm/main.py"
B_scalings = np.linspace(-1.0,1.0,5)
spin_pairs = [(0,1),(0,2)]
SSC_type = 'Z components'

Next, we define a function to prepare the input files' configuration for all the requested multiple runs within separate sub-directories.

In [42]:
def prepare_config_files(B_scalings: 'np.ndarray') -> 'np.ndarray' :
    '''
    Creates as many subdirectories as requested by the user (i.e. =B_scalings.shape[0])) and
    prepares the input files for the associated calculations. The configuration files are indeed 
    modified in order to set up a distinct single-run pylasm execution per subdirectory.
    It also returns the external magnetic field specified within the original configurational file.
    
    Args:
        B_scalings (np.ndarray): Set of the scaling factors for the specified B_field.
    '''
    # Loop over all the given scaling factors
    old_B_field = np.zeros(3)
    for i in range(B_scalings.shape[0]) :
        
        # Create the current subdirectory if needed
        src_dir = '.'
        dest_dir = f'./run_{i}'
        if not os.path.exists(dest_dir) :
            os.mkdir(dest_dir)
        
        # Copy all the files from the working directory to the current subdirectory
        for filename in os.listdir(src_dir):
            src_file = os.path.join(src_dir,filename)
            dest_file = os.path.join(dest_dir,filename)
            if os.path.isfile(src_file) :
                shutil.copy(src_file,dest_file)
        
        # Read the new config file and modify a specific line
        new_config_file = f'run_{i}/init_config.ini'
        with open(new_config_file,'r') as file:
            lines = file.readlines()

        # Loop over the lines
        for n in range(len(lines)) :
            if 'B_field' in lines[n] :
                
                # Extract the old B_field from the current line
                lines[n] = lines[n].replace('B_field','')
                lines[n] = lines[n].replace('=','')
                lines[n] = lines[n].replace('\n','')
                n_empty_spaces = lines[n].count(' ')
                for m in range(n_empty_spaces) :
                    lines[n] = lines[n].replace(' ','')
                old_B_field = np.array(ast.literal_eval(lines[n]))
                
                # Generate the new B_field and update the line
                new_B_field = B_scalings[i]*old_B_field
                lines[n] = f'B_field = {new_B_field.tolist()}\n'
                break
            
            # Raise a ValueError exception if no B_field value is explicitly specified
            if n==len(lines)-1 :
                raise ValueError('No B_field value was found within the given configuration file.')

        # Write the modified lines back to the file
        with open(new_config_file,'w') as file:
            file.writelines(lines)
        
    return old_B_field

Hence we proceed by the implementation of two similar plotting functions, which take care of both reading the output files of interest and generating the final plots using matplotlib package.  

In [43]:
def plot_M_vs_Bfield(B_scalings: 'np.ndarray',old_B_field: 'np.ndarray') -> None :
    '''
    Plots the magnetization strength as a function of the external magnetic field in terms of the given scaling factors.
    
    Args:
        B_scalings (np.ndarray): Set of the chosen scaling factors for the external magnetic field;
        old_B_field (np.ndarray): Reference 3D vector for external magnetic field.
    '''
    # Read the magnetization values from the output files
    M_values = np.zeros(B_scalings.shape[0])
    for i in range(B_scalings.shape[0]) :
        with open(f'./run_{i}/SPIN_OUT.json','r') as file :
            data = json.load(file)
            M_values[i] += data['Magnetization Modulus']
    
    # Set the default font family and size
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 14

    # Initialize the figure and add some aesthetics
    fig, axs = plt.subplots(figsize=(10,6))
    axs.set_title('Magnetization vs Magnetic field',fontdict={'fontsize': 20})
    axs.set_xlabel(r'Scaling factor for $\vec{B} \propto $'+f'{np.round(old_B_field,2).tolist()}',fontdict={'fontsize': 16})
    axs.set_ylabel('Magnetization',fontdict={'fontsize': 16})
    
    # Plot and save
    axs.plot(B_scalings,M_values,'ro-',markersize=8,markerfacecolor='white')
    plt.grid(linestyle='--',linewidth=0.5)
    plt.savefig('M_vs_Bfield.png')
    plt.show()

def plot_SSC_vs_Bfield(B_scalings: 'np.ndarray',old_B_field: 'np.ndarray',spin_pairs: list,SSC_type: str) -> None :
    '''
    Plots the spin-spin correlation (SSC) value for the given spin pairs as a function of the external magnetic field
    in terms of the given scaling factors.
    
    Args:
        B_scalings (np.ndarray): Set of the chosen scaling factors for the external magnetic field;
        old_B_field (np.ndarray): Reference 3D vector for external magnetic field;
        spin_pairs (list): List of 2-dim. tuples indicating the spin-spin correlation values to extract from the output files;
        SSC_type (str): String for the type of spin-spin correlations the user is interested in.
    
    Note:
        The only accepted values for SSC_type are 'X components','Y components','Z components' or 'Total'.
    '''
    # Raise an exception if the SSC_type value is not allowed
    if SSC_type not in ['X components','Y components','Z components','Total'] :
        raise ValueError(f"{SSC_type} is not an allowed value for SSC_type. Choose betweeen 'X components', 'Y components', 'Z components' or 'Total'.")
    
    # Read the SSC values from the output files
    SSC_values = [np.zeros(B_scalings.shape[0]) for i in range(len(spin_pairs))]
    for i in range(len(spin_pairs)) :
        for j in range(B_scalings.shape[0]) :
            with open(f'./run_{j}/SPIN_OUT.json','r') as file :
                data = json.load(file)
                SSC_values[i][j] += data['Spin-Spin Corr. Matrices'][SSC_type][f'Spins {spin_pairs[i][0]}-{spin_pairs[i][1]}']
    
    # Set the default font family and size
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 14

    # Initialize the figure and add some aesthetics
    fig, axs = plt.subplots(figsize=(10,6))
    axs.set_title('Spin-spin correlation vs Magnetic field',fontdict={'fontsize': 20})
    axs.set_xlabel(r'Scaling factor for $\vec{B} \propto $'+f'{np.round(old_B_field,3).tolist()}',fontdict={'fontsize': 16})
    axs.set_ylabel('Spin-spin correlation',fontdict={'fontsize': 16})
    
    # Plot and save
    for i in range(len(spin_pairs)) :
        label = ''
        match SSC_type:
            case 'X components':
                label += r'$S_x^{(i)} \cdot S_x^{(j)}$'
            case 'Y components':
                label += r'$S_y^{(i)} \cdot S_y^{(j)}$'
            case 'Z components':
                label += r'$S_z^{(i)} \cdot S_z^{(j)}$'
            case 'Total':
                label += r'$\vec{S}^{(i)} \cdot \vec{S}^{(j)}$'
        label += f' with i={spin_pairs[i][0]},j={spin_pairs[i][1]}'
        axs.plot(B_scalings,SSC_values[i],'o-',markersize=8,markerfacecolor='white',label=label)
    plt.grid(linestyle='--',linewidth=0.5)
    plt.legend(loc='best')
    plt.savefig('SSC_vs_Bfield.png')
    plt.show()

Finally, we include the code for the actual execution of the multiple runs. This section can be intuitively schematized into a 3-steps procedure as follows:

> (1) Sub-directories Preparation → (2) Parallel Executions → (3) Plotting Outcomes

In [ ]:
# (1)
old_B_field = prepare_config_files(B_scalings)

# (2)
if os.path.exists(main_path):
    processes = []
    for i in range(B_scalings.shape[0]) :
        os.chdir(f'./run_{i}')
        process = subprocess.Popen(['python', main_path])
        processes.append(process)
        os.chdir(f'..')
    for process in processes :
        process.wait()
else:
    raise FileNotFoundError(f"The script at {main_path} does not exist.")

# (3)
plot_M_vs_Bfield(B_scalings,old_B_field)
plot_SSC_vs_Bfield(B_scalings,old_B_field,spin_pairs,SSC_type)